In [6]:
import os
import numpy as np

In [27]:
import pandas as pd
a = pd.read_excel('C:\AutomaticTrain\State.xlsx')
game_name = 'TLMN'
all_level = [int(level) for level in a[game_name][0].strip('][').split(',')]
for level, percent in enumerate(all_level):
    if percent == 0:
        break


0

In [8]:
def CreateFolder(game_name, time_run_game):
    player = 'Agent'  #Tên folder của người chơi
    path_data = f'Agent/{player}/Data'
    if not os.path.exists(path_data):
        os.mkdir(path_data)
    path_save_player = f'Agent/{player}/Data/{game_name}_{time_run_game}/'
    if not os.path.exists(path_save_player):
        os.mkdir(path_save_player)
    return path_save_player

from setup import game_name, time_run_game
path_save_player = CreateFolder(game_name, time_run_game)

per = file_per()
for time in range(3):
    win, per = numba_main_2(Agent, 10000, per, 0)
    print('Lúc train')
    np.save(f'{path_save_player}Train.npy',per)
np.save(f'{path_save_player}Train.npy',per)
perx_Khanh_save = list(np.load(f'{path_save_player}Train.npy',allow_pickle=True))
print('Lúc test')
win, per = numba_main_2(Agent, 1000, per, 0)
print(win)

Lúc train
Lúc train
Lúc train
Lúc test
660


In [ ]:
from base.TLMN_v2.env import *
from Agent.Agent.Agent_player import Agent, file_per

file_per()

In [2]:
getActionSize()

NameError: name 'getActionSize' is not defined